# Face detection and recognition training pipeline

The following example illustrates how to fine-tune an InceptionResnetV1 model on your own dataset. This will mostly follow standard pytorch training patterns.

In [ ]:
!pip install facenet-pytorch

     |████████████████████████████████| 1.9 MB 8.2 MB/s 


In [ ]:
from facenet_pytorch import MTCNN, InceptionResnetV1, fixed_image_standardization, training
import torch
from torch.utils.data import DataLoader, SubsetRandomSampler
from torch import optim
from torch.optim.lr_scheduler import MultiStepLR
from torch.utils.tensorboard import SummaryWriter
from torchvision import datasets, transforms
import numpy as np
import os

In [ ]:
!unzip "./lfw_lite.zip" 

Archive:  ./lfw_lite.zip
  End-of-central-directory signature not found.  Either this file is not
  a zipfile, or it constitutes one disk of a multi-part archive.  In the
  latter case the central directory and zipfile comment will be found on
  the last disk(s) of this archive.
unzip:  cannot find zipfile directory in one of ./lfw_lite.zip or
        ./lfw_lite.zip.zip, and cannot find ./lfw_lite.zip.ZIP, period.


In [3]:
import tarfile

with tarfile.open('lfw_lite.zip.zip') as f:
  f.extractall('.')

ReadError: ignored

#### Define run parameters

The dataset should follow the VGGFace2/ImageNet-style directory layout. Modify `data_dir` to the location of the dataset on wish to finetune on.

In [ ]:
data_dir = './lfw_aug_cleaned'
os.mkdir('lfw_aug_cleaned_cropped')
batch_size = 32
epochs = 8
workers = 0 if os.name == 'nt' else 5

#### Determine if an nvidia GPU is available

In [ ]:
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
print('Running on device: {}'.format(device))

Running on device: cuda:0


#### Define MTCNN module

See `help(MTCNN)` for more details.

In [ ]:
mtcnn = MTCNN(
    image_size=160, margin=0, min_face_size=20,
    thresholds=[0.6, 0.7, 0.7], factor=0.709, post_process=True,
    device=device
)

In [ ]:
import os

#### Perfom MTCNN facial detection

Iterate through the DataLoader object and obtain cropped faces.

In [ ]:
dataset = datasets.ImageFolder(data_dir, transform=transforms.Resize((512, 512)))
dataset.samples = [
    (p, p.replace(data_dir, data_dir + '_cropped'))
        for p, _ in dataset.samples
]
        
loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    collate_fn=training.collate_pil
)

for i, (x, y) in enumerate(loader):
    mtcnn(x, save_path=y)
    print('\rBatch {} of {}'.format(i + 1, len(loader)), end='')
    
# Remove mtcnn to reduce GPU memory usage
del mtcnn

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))
/usr/local/lib/python3.7/dist-packages/torch/nn/functional.py:780: UserWarning: Note that order of the arguments: ceil_mode and return_indices will changeto match the args list in nn.MaxPool2d in a future release.
  warnings.warn("Note that order of the arguments: ceil_mode and return_indices will change"
/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/utils/detect_face.py:183: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different l

Batch 2 of 1121

/usr/local/lib/python3.7/dist-packages/facenet_pytorch/models/mtcnn.py:340: VisibleDeprecationWarning: Creating an ndarray from ragged nested sequences (which is a list-or-tuple of lists-or-tuples-or ndarrays with different lengths or shapes) is deprecated. If you meant to do this, you must specify 'dtype=object' when creating the ndarray.
  probs = np.array(probs)


Batch 1121 of 1121

#### Define Inception Resnet V1 module

See `help(InceptionResnetV1)` for more details.

In [ ]:
resnet = InceptionResnetV1(
    classify=True,
    pretrained='vggface2',
    num_classes=len(dataset.class_to_idx)
).to(device)

  0%|          | 0.00/107M [00:00<?, ?B/s]

#### Define optimizer, scheduler, dataset, and dataloader

In [ ]:
optimizer = optim.Adam(resnet.parameters(), lr=0.001)
scheduler = MultiStepLR(optimizer, [5, 10])

trans = transforms.Compose([
    np.float32,
    transforms.ToTensor(),
    fixed_image_standardization
])
dataset = datasets.ImageFolder(data_dir + '_cropped', transform=trans)
img_inds = np.arange(len(dataset))
np.random.shuffle(img_inds)
train_inds = img_inds[:int(0.8 * len(img_inds))]
val_inds = img_inds[int(0.8 * len(img_inds)):]

train_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(train_inds)
)
val_loader = DataLoader(
    dataset,
    num_workers=workers,
    batch_size=batch_size,
    sampler=SubsetRandomSampler(val_inds)
)

/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


#### Define loss and evaluation functions

In [ ]:
loss_fn = torch.nn.CrossEntropyLoss()
metrics = {
    'fps': training.BatchTimer(),
    'acc': training.accuracy
}

#### Train model

In [ ]:
writer = SummaryWriter()
writer.iteration, writer.interval = 0, 10

print('\n\nInitial')
print('-' * 10)
resnet.eval()
training.pass_epoch(
    resnet, loss_fn, val_loader,
    batch_metrics=metrics, show_running=True, device=device,
    writer=writer
)

for epoch in range(epochs):
    print('\nEpoch {}/{}'.format(epoch + 1, epochs))
    print('-' * 10)

    resnet.train()
    training.pass_epoch(
        resnet, loss_fn, train_loader, optimizer, scheduler,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

    resnet.eval()
    training.pass_epoch(
        resnet, loss_fn, val_loader,
        batch_metrics=metrics, show_running=True, device=device,
        writer=writer
    )

writer.close()



Initial
----------


/usr/local/lib/python3.7/dist-packages/torch/utils/data/dataloader.py:490: UserWarning: This DataLoader will create 5 worker processes in total. Our suggested max number of worker in current system is 2, which is smaller than what this DataLoader is going to create. Please be aware that excessive worker creation might get DataLoader running slow or even freeze, lower the worker number to avoid potential slowness/freeze if necessary.
  cpuset_checked))


Valid |   364/364  | loss:    8.8283 | fps:  212.1215 | acc:    0.0003   

Epoch 1/8
----------
Train |  1453/1453 | loss:    6.0860 | fps:   70.6865 | acc:    0.1656   
Valid |   364/364  | loss:    2.5774 | fps:  213.4602 | acc:    0.5844   

Epoch 2/8
----------
Train |  1453/1453 | loss:    1.3022 | fps:   70.7289 | acc:    0.7921   
Valid |   364/364  | loss:    0.7434 | fps:  213.7783 | acc:    0.8887   

Epoch 3/8
----------
Train |  1453/1453 | loss:    0.4906 | fps:   70.7398 | acc:    0.9102   
Valid |   364/364  | loss:    0.5139 | fps:  214.3150 | acc:    0.9267   

Epoch 4/8
----------
Train |  1453/1453 | loss:    0.3076 | fps:   70.5298 | acc:    0.9353   
Valid |   364/364  | loss:    0.4730 | fps:  212.7683 | acc:    0.9373   

Epoch 5/8
----------
Train |  1453/1453 | loss:    0.2059 | fps:   70.8904 | acc:    0.9533   
Valid |   364/364  | loss:    0.4461 | fps:  212.7064 | acc:    0.9455   

Epoch 6/8
----------
Train |  1453/1453 | loss:    0.0747 | fps:   70.8769 

In [ ]:
print("hi")

hi


In [ ]:
with open('resnet', 'wb') as f:
  torch.save(resnet, f)

In [ ]:
resnet_cpu = resnet.to(device='cpu')

In [ ]:
with open('resnet', 'wb') as f:
  torch.save(resnet_cpu, f)